In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [3]:
# Basic autograd example 1

# create tensors

x = torch.tensor(1., requires_grad = True)
w = torch.tensor(2., requires_grad = True)
b = torch.tensor(3., requires_grad = True)

# Build a computational graph
y = w*x + b

# Compute gradients
y.backward()

# print out the gradients
print(x.grad) 
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


In [5]:
# Basic autograd example 2

#create tensors of shape (10, 3) and (10, 2)
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer 
linear = nn.Linear(3, 2)
print('w: ', linear.weight)
print('b: ',linear.bias)

# Build loss function and optimizer
criterion = nn.MSELoss()
optimizer =  torch.optim.SGD(linear.parameters(), lr = 0.01)

# Forward pass
pred = linear(x)

# Compute loss
loss = criterion(pred, y)
print('loss: ', loss.item())

#Backward pass
loss.backward()

#print out gradients
print('dL/dw: ', linear.weight.grad)
print('dL/db: ', linear.bias.grad)

# 1-step gradient descent
optimizer.step()

# you can also perform gradient descent at low level
# linear.weight.data.sub(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# print out the loss after 1-step gradient descent
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

w:  Parameter containing:
tensor([[-0.0670, -0.4310, -0.2726],
        [ 0.1649,  0.3939, -0.1413]], requires_grad=True)
b:  Parameter containing:
tensor([-0.2304, -0.1785], requires_grad=True)
loss:  0.8109996914863586
dL/dw:  tensor([[-0.5828, -0.5291, -0.6683],
        [ 0.0165, -0.2694, -0.4347]])
dL/db:  tensor([-0.0163, -0.4687])
loss after 1 step optimization:  0.7956877946853638


In [7]:
# 3. Loading data from numpy 

# Create numpy array
x = np.array([[1,2], [3,4]])

# Convert the numpy array to a torch tensor
y = torch.from_numpy(x)

# Convert the torch tensor to numpy array
z = y.numpy()

In [8]:
# 4. Input pipeline

# Download and construct CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root = '../../data/',
                                            train = True,
                                            transform = transforms.ToTensor(),
                                            download = True)

# Fetch one data pair ( read data from disk)
image, label = train_dataset[0]
print(image.size())
print(label)

# Data loader (this provide queues and threads in a very simple way)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size = 64,
                                          shuffle = True)

# When iteration strats , queue and thread start to load data from files
data_iter = iter(train_loader)

# Mini-batch images and labels
images, labels = data_iter.next()

# Actual usage of the data loader is as below 
for images, labels in train_loader:
    # Training code should be written here
    pass

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../../data/cifar-10-python.tar.gz to ../../data/
torch.Size([3, 32, 32])
6


In [12]:
# Input pipeline for custom dataset

# we should build our custom dataset as below

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of files names
        pass
    
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file(e.g. using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (e.g. torchvision.Transform)
        # 3. Return a data pair (e.g. image and label)
        pass
    def __len__(self):
        # We should change from 0 to the total size of the dataset
        return 0
    
# We can then use the prebuilt data loader
custom_dataset =  CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                          batch_size = 64,
                                          shuffle = True)

In [10]:
# 6. Pretrained model

# Download and load the pretrained ResNet-18
resnet = torchvision.models.resnet18(pretrained=True)

# If we want to finetune only the top layer of the model , set as below
for param in resnet.parameters():
    param.requires_grad = False
    
#  Replace the top layer for finetuning
resnet.fc = nn.Linear(resnet.fc.in_features, 100) # 100 is an example

# Forward pass
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print(outputs.size()) # (64,100)



Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/cloudcraftz/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

torch.Size([64, 100])


In [11]:
# 7. Save and load the model
torch.save(resnet, 'resnet_model.ckpt')
model = torch.load('resnet_model.ckpt')

# save and load the only model parameters (recommended)
torch.save(resnet.state_dict(), 'resnet_params.ckpt')
resnet.load_state_dict(torch.load('resnet_params.ckpt'))

<All keys matched successfully>